In [ ]:
!pip install findspark

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext, SparkConf

In [3]:
conf = SparkConf().setMaster('local[*]').setAppName('UnStructured Analysis')
sc = SparkContext(conf = conf)
sc.setLogLevel('ERROR')
sc

23/06/26 10:40:52 WARN Utils: Your hostname, OMEN resolves to a loopback address: 127.0.1.1; using 172.19.181.52 instead (on interface eth0)
23/06/26 10:40:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/26 10:40:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=UnStructured Analysis>

In [4]:
sc.defaultParallelism

16

In [5]:
rdd = sc.textFile('file:///home/deepak/datasets/ghtorrent-logs.txt.gz')

In [6]:
rdd.getNumPartitions()

1

In [7]:
from pyspark import StorageLevel
rdd = rdd.persist(StorageLevel.MEMORY_AND_DISK)

In [8]:
print('The number of records:', rdd.count())

The number of records: 9669788


In [9]:
rdd.takeSample(False, 20, 1234)

['DEBUG, 2017-03-23T12:00:58+00:00, ghtorrent-19 -- ghtorrent.rb: Repo ccurro/tiefvision exists',
 'DEBUG, 2017-03-23T14:07:36+00:00, ghtorrent-20 -- retriever.rb: issues drone/drone -> 1604 exists',
 'INFO, 2017-03-23T10:07:20+00:00, ghtorrent-29 -- ghtorrent.rb: Added issue_event xcat2/xcat-core -> 2703/xcat2/xcat-core -> 2703/1008161947',
 'INFO, 2017-03-23T09:17:58+00:00, ghtorrent-19 -- api_client.rb: Successful request. URL: https://api.github.com/repos/yuwangying/PyCTP/issues/17/labels?per_page=100, Remaining: 2192, Total: 85 ms',
 'DEBUG, 2017-03-24T13:05:50+00:00, ghtorrent-47 -- ghtorrent.rb: Repo smcintosh881/343-accounting exists',
 'DEBUG, 2017-03-23T10:12:50+00:00, ghtorrent-7 -- ghtorrent.rb: Transaction committed (30 ms)',
 'DEBUG, 2017-03-23T10:42:40+00:00, ghtorrent-14 -- retriever.rb: Commit derlio/JieCaoVideoPlayer -> bd4b967d7e50256fab2e74f531013649d04e0246 exists',
 'DEBUG, 2017-03-23T10:12:52+00:00, ghtorrent-7 -- ghtorrent.rb: User mithro exists',
 'DEBUG, 2017-

In [9]:
import re
def collect_words(line):
    return re.compile('\w+').findall(line.lower())

In [14]:
print(collect_words('Hello There! Who are you??'))

['hello', 'there', 'who', 'are', 'you']


In [ ]:
# Number of lines with both transaction and repo in the lines

In [20]:
rdd_transactions = rdd.filter(lambda line: 'transaction' in collect_words(line))
rdd_repos = rdd.filter(lambda line: 'repo' in collect_words(line))
rdd_intersection = rdd_transactions.intersection(rdd_repos)
rdd_intersection.count()

19

In [21]:
rdd_intersection.collect()

['INFO, 2017-03-23T09:13:16+00:00, ghtorrent-9 -- ghtorrent.rb: Repo xuminwlt/tcc-transaction is a fork of changmingxie/tcc-transaction',
 'DEBUG, 2017-03-23T09:13:17+00:00, ghtorrent-9 -- retriever.rb: Repo xuminwlt -> tcc-transaction exists',
 'DEBUG, 2017-03-23T09:13:17+00:00, ghtorrent-9 -- ghtorrent.rb: Repo xuminwlt/tcc-transaction exists',
 'INFO, 2017-03-23T09:13:17+00:00, ghtorrent-9 -- ghtorrent.rb: Added repo xuminwlt/tcc-transaction',
 'INFO, 2017-03-23T09:13:16+00:00, ghtorrent-9 -- retriever.rb: Added repo xuminwlt -> tcc-transaction',
 'DEBUG, 2017-03-23T09:13:28+00:00, ghtorrent-9 -- ghtorrent.rb: Repo xuminwlt/tcc-transaction exists',
 'DEBUG, 2017-03-23T09:13:17+00:00, ghtorrent-9 -- ghtorrent.rb: Repo changmingxie/tcc-transaction exists',
 'DEBUG, 2017-03-23T09:13:16+00:00, ghtorrent-9 -- ghtorrent.rb: Repo changmingxie/tcc-transaction exists',
 'DEBUG, 2017-03-23T09:13:27+00:00, ghtorrent-9 -- ghtorrent.rb: Repo xuminwlt/tcc-transaction exists',
 'DEBUG, 2017-03-23T

In [ ]:
# Number of lines including web link for WARN log level

In [23]:
def getUrl(line):
    return re.findall(r'http[s]?://(?:[-\w.]|(?:%[\da-zA-Z]{2,}))+', line)
getUrl('Hello There! https://google.com')

['https://google.com']

In [24]:
rdd.filter(lambda line: line.split(',')[0] == 'WARN').filter(lambda line: len(getUrl(line)) > 0).count()

95329

In [ ]:
# What is the mot active downloader id for failed connections?

In [25]:
rdd.take(10)

['INFO, 2017-03-22T20:11:49+00:00, ghtorrent-31 -- ghtorrent.rb: Added pullreq_commit 244eeac28bf419642d5d5c3b388bd2999c8c72e6 to tgstation/tgstation -> 25341',
 'DEBUG, 2017-03-23T11:15:14+00:00, ghtorrent-30 -- retriever.rb: Commit mzvast/FlappyFrog -> 80bf5c5fde7be6274a2721422f4d9a773583f73c exists',
 'DEBUG, 2017-03-22T20:15:48+00:00, ghtorrent-35 -- ghtorrent.rb: Parent af8451e16e077f7c6cae3f98bf43bffaca562f88 for commit 2ef393531a3cfbecc69f17d2cedcc95662fae1e6 exists',
 'DEBUG, 2017-03-24T12:29:50+00:00, ghtorrent-49 -- ghtorrent.rb: Parent cf060bf3b789ac6391b2f7c1cdc34191c2bc773d for commit 8c924c1115e1abddcaddc27c6e7fd5806583ea90 exists',
 'DEBUG, 2017-03-23T09:00:44+00:00, ghtorrent-8 -- retriever.rb: Commit iamtheanon/d3 -> a7caf9375fe14d7235562af541fe9decf499bbfb exists',
 'DEBUG, 2017-03-24T10:52:47+00:00, ghtorrent-50 -- ghtorrent.rb: Repo alyuev/urantia-study-edition exists',
 'DEBUG, 2017-03-23T11:02:10+00:00, ghtorrent-45 -- retriever.rb: Commit heycalmdown/node-conflue

In [26]:
rdd_failed = rdd.filter(lambda line: 'failed' in collect_words(line))
rdd_active_ids = rdd_failed.map(lambda line: (line.replace('--',',').split(',')[2].split('-')[1], 1) )

In [27]:
rdd_active_ids.take(10)

[('13 ', 1),
 ('13 ', 1),
 ('13 ', 1),
 ('39 ', 1),
 ('13 ', 1),
 ('21 ', 1),
 ('13 ', 1),
 ('21 ', 1),
 ('20 ', 1),
 ('13 ', 1)]

In [28]:
rdd_result = rdd_active_ids.groupByKey(4).mapValues(sum).sortBy(lambda x: x[1], ascending=False)

In [29]:
rdd_result.first()

('13 ', 79654)

In [30]:
rdd_active_ids.reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending=False).first()

('13 ', 79654)

In [ ]:
# What is the most active repository?

In [33]:
def get_repo(line):
    try:
        return re.compile('\w+/\w+').findall(line.lower())[0]
    except:
        return None
get_repo('DEBUG, 2017-03-24T10:52:47+00:00, ghtorrent-50 -- ghtorrent.rb: Repo alyuev/urantia-study-edition exists')

'alyuev/urantia'

In [38]:
rdd.map(lambda line: get_repo(line)).filter(lambda repo: repo is not None).map(lambda repo: (repo,1)).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending=False).first()

('com/repos', 884219)